In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from pprint import pprint
from pathlib import Path
from llm_assessor import (
    ranged_error
)

load_dotenv(override=True)

ERROR:root:Failed to update token costs. Using static costs.
/Users/junaid/.virtualenvs/audenshaw_exam_validation/lib/python3.11/site-packages/tokencost/constants.py:61: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  TOKEN_COSTS = TOKEN_COSTS_STATIC


True

In [2]:
# Set working directory - Not required if using Jupyter outside of VScode
workdir = os.environ["workdir"]
os.chdir(workdir)
os.getcwd()

'/Users/junaid/Developer/audenshaw_exam_validation'

In [3]:
# Load Dataframe
data_dir = Path("./validation_results")

teacher_marked_questions = pd.read_csv(data_dir / "processed_data" / "student_answers_llm_graded.csv")

In [4]:
teacher_marked_questions.head()

,subject_id,question_id,question_type,student_id,question_text,mark_scheme_text,context,answer_text,awarded_marks,total_marks,...,topic_id,answer_scanned_image,start_time,llm_graded_answer,llm_graded_answer_token_costing,llm_awarded_marks,llm_awarded_marks_token_costing,llm_mark_hitrate,end_time,elapsed_time_in_seconds
0,aqa_history,1.0,hs_explain,5000,Source A is critical of the Bolsheviks during ...,Level_2:\n description: \n - Developed ana...,Source A: A British cartoon entitled ‘Betraye...,Source A is critical of the \nBolsheviks as in...,1,4,...,NaN,NaN,1.721054e+09,### Step-by-Step Grading Explanation\n\n#### 1...,"{'burn_in_runs': 2, 'model': 'gpt-4o', 'prompt...",2,"{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt...",False,1.721054e+09,39.719050
1,aqa_history,2.0,hs_explain,5000,How useful are Sources B and C to an historian...,Level_4:\n description: \n - Complex evalu...,Source B A picture published in a British news...,\nI think Sources\nB and C are very useful to ...,4,12,...,NaN,NaN,1.721054e+09,### Step-by-Step Grading Explanation\n\n#### 1...,"{'burn_in_runs': 2, 'model': 'gpt-4o', 'prompt...",4,"{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt...",True,1.721054e+09,27.226788
2,aqa_history,3.0,hs_analyse,5000,Write an account of how the Schlieffen Plan le...,Level_4:\n description: \n - Answer is pre...,NaN,The Schlieffen plan led to problems in 1914 as...,0,8,...,NaN,NaN,1.721054e+09,### Step-by-Step Grading Explanation\n\n#### 1...,"{'burn_in_runs': 2, 'model': 'gpt-4o', 'prompt...",1,"{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt...",False,1.721054e+09,22.838951
3,aqa_history,4.1,hs_judgement,5000,The Alliance System was the main cause of the ...,Level_4:\n description: \n - Complex expla...,NaN,I agree with this statement as I believe the ...,8,16,...,NaN,NaN,1.721054e+09,### Step-by-Step Grading Explanation\n\n#### 1...,"{'burn_in_runs': 2, 'model': 'gpt-4o', 'prompt...",7,"{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt...",False,1.721054e+09,36.361025
4,aqa_history,4.2,hs_spag,5000,The Alliance System was the main cause of the ...,Level_4:\n description: \n - High Performa...,NaN,I agree with this statement as I believe the ...,3,4,...,NaN,NaN,1.721054e+09,"Based on the provided Mark Scheme, let's asses...","{'burn_in_runs': 2, 'model': 'gpt-4o', 'prompt...",3,"{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt...",True,1.721054e+09,20.125219


# Compute performance statistics

In [5]:
teacher_marked_questions_reduced = teacher_marked_questions[~teacher_marked_questions.question_type.isin(["bs_mcq", "bs_computation"])]

In [6]:
# Hitrate
## Mean overall
mean_hitrate = np.mean(teacher_marked_questions_reduced.llm_mark_hitrate)

## Mean by question type
mean_hitrate_by_question_type = teacher_marked_questions_reduced.groupby(by=["question_type"], as_index = False).agg({"question_id": "count", "llm_mark_hitrate": ["sum", "mean"]}).rename(columns={"question_id": "number_of_questions", "llm_mark_hitrate": "proportion_of_questions_covered"})
mean_hitrate_by_question_type.columns = ["_".join(col_name).rstrip('_') for col_name in mean_hitrate_by_question_type.columns]

print(f"Overall Mean:  {mean_hitrate:.2f}")
mean_hitrate_by_question_type

Overall Mean:  0.30


,question_type,number_of_questions_count,proportion_of_questions_covered_sum,proportion_of_questions_covered_mean
0,hs_analyse,4,0,0.000
1,hs_explain,8,3,0.375
2,hs_judgement,4,1,0.250
3,hs_spag,4,2,0.500


In [7]:
# Hitrate by Question ID
teacher_marked_questions_reduced.head()

question_hit_rate = teacher_marked_questions_reduced.groupby(by=['question_id', "question_type", "total_marks"], as_index=False).agg({"llm_mark_hitrate": ["count", "sum", "mean"]})
question_hit_rate.columns = ["question_id", "question_type", "total_marks", "number_of_questions", "hitrate_frequency", "mean_hitrate"]

question_hit_rate

,question_id,question_type,total_marks,number_of_questions,hitrate_frequency,mean_hitrate
0,1.0,hs_explain,4,4,1,0.25
1,2.0,hs_explain,12,4,2,0.50
2,3.0,hs_analyse,8,4,0,0.00
3,4.1,hs_judgement,16,4,1,0.25
4,4.2,hs_spag,4,4,2,0.50


In [9]:
# Mean Absolute Error
teacher_marked_questions_reduced["marks_mean_absolute_error"] = teacher_marked_questions_reduced.apply(lambda row: ranged_error(x=row['llm_awarded_marks'], range_of_values=[row['awarded_marks']]), axis=1)

# Overall
marks_mea = np.mean(teacher_marked_questions_reduced.marks_mean_absolute_error)

## Mean by question type
marks_mea_by_question_type = teacher_marked_questions_reduced.groupby(by=["question_type"], as_index = False).agg({"question_id": "count", "marks_mean_absolute_error": "mean"}).rename(columns={"question_id": "number_of_questions"})

print(f"Overall Mean:  {marks_mea:.2f}")
marks_mea_by_question_type

Overall Mean:  1.60


,question_type,number_of_questions,marks_mean_absolute_error
0,hs_analyse,4,2.750
1,hs_explain,8,0.875
2,hs_judgement,4,2.250
3,hs_spag,4,1.250


In [10]:
# Mean Absolute Error by question deviation
question_type_mark_deviations = teacher_marked_questions_reduced.groupby(by=["question_type", "marks_mean_absolute_error"], as_index = False).agg({"question_id": "count"})
question_type_totals = teacher_marked_questions_reduced.groupby(by=["question_type"], as_index = False).agg({"question_id": "count"})

question_type_mark_deviation_joined = pd.merge(question_type_mark_deviations, question_type_totals, on = "question_type", how="left")
question_type_mark_deviation_joined['questions_proportion'] = question_type_mark_deviation_joined.apply(lambda x: x['question_id_x']/ x['question_id_y'], axis = 1)

question_type_mark_deviation_joined = question_type_mark_deviation_joined.rename(columns={"question_id_x": "number_of_questions", "question_id_y": "number_of_questions_by_question_type"})
question_type_mark_deviation_joined

,question_type,marks_mean_absolute_error,number_of_questions,number_of_questions_by_question_type,questions_proportion
0,hs_analyse,1,1,4,0.250
1,hs_analyse,2,2,4,0.500
2,hs_analyse,6,1,4,0.250
3,hs_explain,0,3,8,0.375
4,hs_explain,1,3,8,0.375
5,hs_explain,2,2,8,0.250
6,hs_judgement,0,1,4,0.250
7,hs_judgement,1,1,4,0.250
8,hs_judgement,3,1,4,0.250
9,hs_judgement,5,1,4,0.250


In [11]:
overall_stats = pd.DataFrame({"mean_hitrate": [np.round(mean_hitrate, 2)], "marks_mean_abolute_error": [np.round(marks_mea, 2)], "number_of_questions": [teacher_marked_questions.shape[0]], "elapsed_time_in_minutes": [np.round( np.sum(teacher_marked_questions.elapsed_time_in_seconds), 0) / 60]})
overall_stats

,mean_hitrate,marks_mean_abolute_error,number_of_questions,elapsed_time_in_minutes
0,0.3,1.6,20,10.033333


In [ ]:
# Save validation statistics tables
savedir = Path(workdir) / "validation_results" / "business_studies" / year / "validation_statistics"

Path(savedir).mkdir(parents=True, exist_ok=True)

In [ ]:
teacher_marked_questions.to_csv(savedir / "examiner_llm_comparison.csv", index=False)

In [ ]:
overall_stats.to_csv(savedir / "mean_hitrate.csv", index=False)
mean_hitrate_by_question_type.to_csv(savedir / "mean_hitrate_by_question_type.csv", index=False)
marks_mea_by_question_type.to_csv(savedir / "mea_marks_by_question_type.csv", index=False)
question_type_mark_deviation_joined.to_csv(savedir / "question_type_marks_deviation.csv", index=False)
question_hit_rate.to_csv(savedir / "question_hit_rate.csv", index=False)